In [106]:
import numpy as np
import pandas as pd
import warnings
import copy
import random
from scipy.special import softmax
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings("ignore")

### Read In Data

In [2]:
training_set = pd.read_csv("./fashion-mnist_train.csv")
training_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Columns: 785 entries, label to pixel784
dtypes: int64(785)
memory usage: 359.3 MB


In [3]:
testing_set = pd.read_csv("./fashion-mnist_test.csv")
testing_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 785 entries, label to pixel784
dtypes: int64(785)
memory usage: 59.9 MB


In [4]:
training_set.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
testing_set.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,9,8,...,103,87,56,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,34,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,14,53,99,...,0,0,0,0,63,53,31,0,0,0
3,2,0,0,0,0,0,0,0,0,0,...,137,126,140,0,133,224,222,56,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Split into class labels and data points

In [8]:
x_train = training_set.iloc[:, 1:]
y_train = training_set.iloc[:, 0]

In [10]:
x_test = testing_set.iloc[:, 1:]
y_test = testing_set.iloc[:, 0]

### Shuffle Data

In [13]:
np.random.seed(42)

In [14]:
# convert to numpy objects
x_train = x_train.to_numpy()
y_train = y_train.to_numpy()
x_test = x_test.to_numpy()
y_test = y_test.to_numpy()

In [17]:
# shuffle training data
arr = np.arange(len(x_train))
np.random.shuffle(arr)
x_train = x_train[arr]
y_train = y_train[arr]

In [19]:
# shuffle testing data
arr = np.arange(len(x_test))
np.random.shuffle(arr)
x_test = x_test[arr]
y_test = y_test[arr]

### Sample Data

In [20]:
# take the first 10K points as the training data
x_train = x_train[:10000].copy()
y_train = y_train[:10000].copy()

In [21]:
# take the first 5K points as the test data
x_test = x_test[:5000].copy()
x_test = x_test[:5000].copy()

# Multiclass Logistic Regression

In [137]:
def LogReg(x_train, y_train, eta, epsilon):
    # map x data to d+1 dimensions
    x = []
    for i in range(len(x_train)):
        x.append(np.insert(x_train[i], 0, 1))
    x = np.array(x)
    
    # create a one hot encoding of the target labels
    y = np.zeros( (y_train.size, y_train.max() + 1) )
    y[np.arange(y_train.size), y_train] = 1
    
    t = 0
    
    # create initial weights matrix
    # K rows and d+1 cols
    w = np.zeros((10, 785) )
    
    while(True):
        # make a copy of w
        w_old = copy.deepcopy(w)
    
    
        # iterate through dataset in random order
        r = list(range(len(x)))
        random.shuffle(r)
        for i in r:
            
            # get vector of dot products
            prods = np.dot(w, x[i])
            
            # get the softmax for the K dot products
            sm = softmax(prods)
            
            # claculate gradient and update wieghts
            for j in range(len(w)-1):
                grad = (y[i][j] - sm[j]) * x[i]
                new_w = w[j] + eta * grad
                w[j] = new_w
            
        
        # check the break condition
        acc = 0
        for i in range(len(w)-1):
            n = np.linalg.norm(w[j] - w_old[j])
            acc += n
        
        
        if acc <= epsilon:
            break
        else: 
            t += 1
    
    return w

In [166]:
weights = LogReg(x_train, y_train, 1e-6, 1e-1)

In [169]:
for w in weights:
    print(w)

[ 9.27057056e-05 -9.96335396e-07  2.81458281e-06 -8.92923579e-05
  2.93486997e-04  2.33354582e-04  4.39753795e-04 -9.75006384e-04
  1.30172313e-03  2.99373962e-03 -2.46344988e-04 -4.09800660e-03
  2.47699583e-03 -4.19907494e-03 -8.56723509e-03 -4.58531310e-03
 -2.46530614e-03  3.07159481e-03  1.83164659e-03 -1.80467748e-03
  1.57215149e-03  1.23022159e-03 -1.77064316e-03  2.93445802e-04
 -2.35241911e-04 -8.58977715e-06 -2.62997513e-05 -5.06393778e-05
  2.94719551e-05  1.49454462e-05 -1.02506075e-04  1.09909196e-03
  1.52309566e-03  3.98976902e-04 -7.69413838e-04 -7.53898296e-03
 -3.16661453e-04  2.09671508e-03 -1.65663703e-03  5.72162768e-03
  1.44879443e-03  2.46889180e-03 -1.26399023e-03 -3.91318131e-04
  1.71652199e-03  5.03856196e-03  1.26831450e-03  3.37926227e-03
  1.79021704e-03  7.37996914e-03  4.01502377e-03 -1.72959549e-03
 -2.04363138e-03 -1.27221230e-03  5.61909119e-04 -9.24992693e-05
  1.08611648e-05 -1.78704988e-05 -5.09969343e-04  1.50660401e-03
  1.21552320e-03  1.53464

In [177]:
def predict(w, x_test):
    # augment test data
    x = []
    for i in range(len(x_test)):
        x.append(np.insert(x_test[i], 0, 1))
    x = np.array(x)
    
    predictions = []
    
    # for all test points
    # compute dot products between weights and data points
    # then compute the softmax and take the argmax
    for i in range(len(x)):
        prods = np.dot(w, x[0])
        sm = softmax(prods)
        index = np.argmax(sm)
        predictions.append(index)
    
    predictions = np.array(predictions)
    return predictions
    
    

In [178]:
y_pred = predict(weights, x_test)

In [179]:
# calculate accuracy
correct = 0
for i in range(len(y_pred)):
    if y_pred[i] == y_test[i]:
        correct += 1

acc_score = correct / len(y_pred)

In [180]:
acc_score

0.1

In [186]:
# computing f1 score
prec=[]
rec=[]

# first compute the class specific accuracies
for i in range(10):
    correct = 0
    total = 0
    for j in range(len(y_pred)):
        if y_pred[j] == i:
            total += 1
            if y_test[j] == i:
                correct += 1
    
    if total == 0:
        prec.append(0)
    else:
        prec.append(correct / total)

    
# next compute the class specific recall
for i in range(10):
    correct = 0
    total = 0
    for j in range(len(y_pred)):
        if y_test[j] == i:
            total += 1
            if y_pred[j] == i:
                correct += 1
    
    rec.append(correct / total)


In [187]:
# next compute the overall precision and recall
overall_prec = np.mean(prec)
overall_recall = np.mean(rec)

# finally compute the f1 score
f1 = (2 * overall_prec * overall_recall) / (overall_prec + overall_recall)

In [188]:
print(f1)

0.01818181818181818
